In [ ]:
import gymnasium as gym
import os
from stable_baselines3 import PPO
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy


In [ ]:
env = gym.make('CartPole-v1', render_mode="rgb_array")

In [ ]:
env.action_space.sample()

In [ ]:
env.reset()
env.step(env.action_space.sample())

In [ ]:
episodes = 5

# random policy
for episode in range(1, episodes+1):
    state = env.reset()
    done1 = False
    done2 = False
    score = 0
    
    while not (done1 or done2):
        env.render()
        action = env.action_space.sample()
        n_state, reward, done1, done2, _ = env.step(action)
        score += reward
    print(f'Episode {episode} Score {score}')
    
env.close()

In [ ]:
log_path = os.path.join('Training', 'Logs')

In [ ]:
if not os.path.exists(log_path):
    os.makedirs(log_path)
env = Monitor(env, log_path)
env = DummyVecEnv([lambda:env])

In [ ]:
model = PPO('MlpPolicy', env, verbose=1, tensorboard_log=log_path)

In [ ]:
model.learn(total_timesteps=20000)

In [ ]:
PPO_Path = os.path.join('Training', 'Models', 'PPO_Model_Reacher')

model.save(PPO_Path)

In [ ]:
model??

In [ ]:
del model

In [ ]:
model = PPO.load(PPO_Path, env=env, verbose=1)

In [ ]:
evaluate_policy(model, env, n_eval_episodes=10, render=True)

In [ ]:
model.predict(env.reset())

In [ ]:
episodes = 5
for episode in range(1, episodes+1):
    obs = env.reset()
    done1 = False
    done2 = False
    score = 0
    
    while not (done1 and done2):
        env.render()
        action, _ = model.predict(obs)
        obs, reward, done1, done2 = env.step(action)
        score += reward
    print(f'Episode {episode} Score {score}')
env.close()

In [ ]:
model.predict??

In [ ]:
training_log_path = os.path.join('Training', 'Logs', 'PPO_5')

In [ ]:
# callback

In [ ]:
from stable_baselines3.common.callbacks import EvalCallback, StopTrainingOnRewardThreshold

In [ ]:
best_model_path = os.path.join('Training', 'BestModels')

In [ ]:
stop_callback = StopTrainingOnRewardThreshold(reward_threshold=10)
# eval_callback triggered after each trainning run, every time in best model, it gona run that stopcallback
eval_callback = EvalCallback(env, callback_on_new_best=stop_callback,best_model_save_path=best_model_path, eval_freq=10000, verbose=1)

In [ ]:
model2 = PPO('MlpPolicy', env, verbose=1, tensorboard_log=log_path)

In [ ]:
model2.learn(total_timesteps=200000, callback=eval_callback)

In [ ]:
# changing policies

In [ ]:
net_arch = dict(pi=[128, 128, 128, 128], vf=[128, 128, 128, 128])

In [ ]:
model3 = PPO('MlpPolicy', env, verbose=1, tensorboard_log=log_path, policy_kwargs={'net_arch': net_arch})

In [ ]:
model3.learn(total_timesteps=20000, callback=eval_callback)

In [ ]:
# use other algo, e.g. DDPG

In [ ]:
from stable_baselines3 import DDPG

In [ ]:
model4 = DDPG('MlpPolicy', env, verbose=1, tensorboard_log=log_path)

In [ ]:
model4.learn(total_timesteps=20000, callback=eval_callback)